# Image rotation with CNN

this is a search and development project to do on the weekends.


my main resourse is this blog post: https://d4nst.github.io/2017/01/12/image-orientation/



In [22]:
targetUrl = 'datasets/streetview'
targetUrlTestData = 'datasets/streetview/testdata'

gmap_images_url = "http://www.cs.ucf.edu/~aroshan/index_files/Dataset_PitOrlManh/images"
DEBUG = True

def dpr(output):
    if (DEBUG):
        print(output)

## Rotate images
#### Get images

In [3]:
import math
# generate filenames
# example 000005_3.jpg
# the postfix can be a number between 0 and 5
# the first part is a simple xml padded to 6 char: '{0:0>6}'.format('132')
fileNames = []
max = 100
for i in range(1, math.floor(max/6)+1):
    baseName = '{0:0>6}'.format(i);
    for postfix in range(6):
        fileNames.append(baseName + "_" + str(postfix) + ".jpg");


In [3]:
fileNames[0:10]

['000001_0.jpg',
 '000001_1.jpg',
 '000001_2.jpg',
 '000001_3.jpg',
 '000001_4.jpg',
 '000001_5.jpg',
 '000002_0.jpg',
 '000002_1.jpg',
 '000002_2.jpg',
 '000002_3.jpg']

### actually download images

In [59]:
import urllib.request

test_train_split = 8
counter = 0

for filename in fileNames:
    print('downloading {0}\n'.format(filename))
    counter += 1
    if (counter % 8 == 0):
        targetDir = targetUrlTestData
    else:
        targetDir = targetUrl
    
    urllib.request.urlretrieve(gmap_images_url + '/' + filename, targetDir + '/' + filename)


downloading 000001_0.jpg

downloading 000001_1.jpg

downloading 000001_2.jpg

downloading 000001_3.jpg

downloading 000001_4.jpg

downloading 000001_5.jpg

downloading 000002_0.jpg

downloading 000002_1.jpg

downloading 000002_2.jpg

downloading 000002_3.jpg

downloading 000002_4.jpg

downloading 000002_5.jpg

downloading 000003_0.jpg

downloading 000003_1.jpg

downloading 000003_2.jpg

downloading 000003_3.jpg

downloading 000003_4.jpg

downloading 000003_5.jpg

downloading 000004_0.jpg

downloading 000004_1.jpg

downloading 000004_2.jpg

downloading 000004_3.jpg

downloading 000004_4.jpg

downloading 000004_5.jpg

downloading 000005_0.jpg

downloading 000005_1.jpg

downloading 000005_2.jpg

downloading 000005_3.jpg

downloading 000005_4.jpg

downloading 000005_5.jpg

downloading 000006_0.jpg

downloading 000006_1.jpg

downloading 000006_2.jpg

downloading 000006_3.jpg

downloading 000006_4.jpg

downloading 000006_5.jpg

downloading 000007_0.jpg

downloading 000007_1.jpg

downloading 

### rotate image in 3 orientation, copy and add some label

labels: https://www.impulseadventure.com/photo/exif-orientation.html

- 1 top, 
- 8 left side, 
- 3 bottom, 
- 6 right side

prepend this label to the filename separted by a "#", eg. `1#000001_5.jpg`



In [4]:
import os
from PIL import Image

ORIENTATION_LABELS = { 'top': 1, 'left':  8, 'bottom': 3, 'right': 6 }


def rotateImage(orientation, fileName, labeledFileName, subdir):
    anglesForOrientation = { ORIENTATION_LABELS['top']: 0, 
                             ORIENTATION_LABELS['left']: 90, 
                             ORIENTATION_LABELS['bottom']: 180, 
                            ORIENTATION_LABELS['right']: 270 }
    
    if not orientation in anglesForOrientation.keys():
        raise Exception('Unknown orienation: ' + orientation)
    
    angle = anglesForOrientation[orientation]
    
    # crop the image to have equal sizes
    targetPath = os.path.join(subdir, labeledFileName)  
    img = Image.open(os.path.join(subdir, fileName))
    size = 0, 0, 1024, 1024
    img.crop(size).rotate(angle).save(targetPath)
                                  
#rotateImage(ORIENTATION_LABELS['bottom'], 1, 2, 3)

In [5]:
# first iterate them and add the appropriate label
import os


def isImage(file):
    return '.jpg' in file

def isProcessed(file):
    return '#' in file

def addLabelToFile(label, file):
    return '{label}#{file}'.format(label = label, file = file)


def rotateImages(fileName, subdir):
    for orientation in ORIENTATION_LABELS.values():
        labeledFileName = addLabelToFile(orientation, fileName)
        rotateImage(orientation, fileName, labeledFileName, subdir)


def processImages(directory):
    for subdir, dirs, files in os.walk(directory):
        for file in files:
            if (not isImage(file)):
                continue
            if (isProcessed(file)):
                continue
            print ('Processing: {0}'.format(os.path.join(subdir, file)))
        
        
            # create the rotations
            rotateImages(file, subdir)
        
            # delete originals
            os.remove(os.path.join(subdir, file))
        

### Actually rotate and label images in the test and train directories

In [64]:
processImages(targetUrl)
processImages(targetUrlTestData)

Processing: datasets/streetview/000013_5.jpg
Processing: datasets/streetview/000015_2.jpg
Processing: datasets/streetview/000013_4.jpg
Processing: datasets/streetview/000011_4.jpg
Processing: datasets/streetview/000015_0.jpg
Processing: datasets/streetview/000008_4.jpg
Processing: datasets/streetview/000015_1.jpg
Processing: datasets/streetview/000011_5.jpg
Processing: datasets/streetview/000013_3.jpg
Processing: datasets/streetview/000011_1.jpg
Processing: datasets/streetview/000015_5.jpg
Processing: datasets/streetview/000008_0.jpg
Processing: datasets/streetview/000008_1.jpg
Processing: datasets/streetview/000015_4.jpg
Processing: datasets/streetview/000011_0.jpg
Processing: datasets/streetview/000013_2.jpg
Processing: datasets/streetview/000013_0.jpg
Processing: datasets/streetview/000011_2.jpg
Processing: datasets/streetview/000008_3.jpg
Processing: datasets/streetview/000008_2.jpg
Processing: datasets/streetview/000013_1.jpg
Processing: datasets/streetview/000014_0.jpg
Processing

# convert images into classified matrices before train / test split

Keras has a fit_generator where the first param is a generator to add datas on the fly (by batches)

https://keras.io/models/sequential/


In [60]:
import os
import numpy as np
from PIL import Image
import re


class trainDataGenerator:
    def __init__(self, image_src_path, batch_size=64):
        self.image_src_path = image_src_path
        self.batch_size = batch_size
        self.input_shape = (1024, 1024, 3)
        self.files = []
        self._getFilenames()
        dpr("number of images: {}".format(len(self.files)))
        self.curr_image_index = -1
        self.batch_X = False
        self.batch_y = False
        self.batch_index = -1

        
    def __iter__(self):
        return self

    def __next__(self):
        # create array to hold the images
        self.batch_X = np.zeros((self.batch_size,) + self.input_shape, dtype='float32')
        dpr("batch_X shape: {}".format(self.batch_X.shape))
        # create array to hold the labels
        self.batch_y = np.zeros(self.batch_size, dtype='int')
        dpr(self.curr_image_index)
        first_image_in_batch = self.curr_image_index + 1
        last_image_in_batch = first_image_in_batch + self.batch_size
        self.batch_index = -1

        self._processFiles(first_image_in_batch,  last_image_in_batch)
        dpr("batch_X shape: {}, batch_y shape: {}".format(self.batch_X.shape, self.batch_y.shape))

        return self.batch_X, keras.utils.to_categorical(self.batch_y, num_classes=None)
    
    def _processFiles(self, first_image_in_batch, last_image_in_batch):
        dpr('first: {}, second: {}'.format(first_image_in_batch, last_image_in_batch))
        for i in range(first_image_in_batch, last_image_in_batch):
            try:
                file_name = self.files[i]
                self.curr_image_index = i
                img = Image.open(file_name)
                arr = np.array(img)
                # image as numpy array
                shape = arr.shape
                dpr('{image} has shape: {shape}'.format(image=file_name, shape=shape))
                # batch_X index is always between 0 and batch size, but index starts at the first image in batch.
                # when the batch is 4, first will be 0, then 4, then 8 

                self.batch_index += 1
                self.batch_X[self.batch_index] = arr
                
                # get the label
                found = re.search(r'([1,8,3,6])#.*?', file_name)

                label = found.group(1)
                dpr('label: {label}'.format(label=label))
                # convert label to categorical
                categs = {'1':1, '8': 2, '3': 3, '6': 4}
                
                
                self.batch_y[self.batch_index] = categs[label]
            
            except FileNotFoundError:
                dpr('cannot open: ' + file_name)
            except IndexError:
                raise StopIteration()
    
    def _getFilenames(self):
        for _, _, files in os.walk(self.image_src_path):
            for file in files:
                 if '.jpg' in file:
                    self.files.append(self.image_src_path + '/' + file)  
                            
                

In [61]:
# train data
for x, y in trainDataGenerator(targetUrlTestData, 4):
    print(x.shape, y.shape)

number of images: 48
batch_X shape: (4, 1024, 1024, 3)
-1
first: 0, second: 4
datasets/streetview/testdata/6#000002_1.jpg has shape: (1024, 1024, 3)
label: 6
datasets/streetview/testdata/3#000008_5.jpg has shape: (1024, 1024, 3)
label: 3
datasets/streetview/testdata/1#000002_1.jpg has shape: (1024, 1024, 3)
label: 1
datasets/streetview/testdata/8#000014_1.jpg has shape: (1024, 1024, 3)
label: 8
batch_X shape: (4, 1024, 1024, 3), batch_y shape: (4,)
(4, 1024, 1024, 3) (4, 5)
batch_X shape: (4, 1024, 1024, 3)
3
first: 4, second: 8
datasets/streetview/testdata/8#000012_5.jpg has shape: (1024, 1024, 3)
label: 8
datasets/streetview/testdata/3#000015_3.jpg has shape: (1024, 1024, 3)
label: 3
datasets/streetview/testdata/6#000004_5.jpg has shape: (1024, 1024, 3)
label: 6
datasets/streetview/testdata/1#000004_5.jpg has shape: (1024, 1024, 3)
label: 1
batch_X shape: (4, 1024, 1024, 3), batch_y shape: (4,)
(4, 1024, 1024, 3) (4, 5)
batch_X shape: (4, 1024, 1024, 3)
7
first: 8, second: 12
dataset

# classifier model definition

keras example for mnist classification: https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py


In [65]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

# number of convolutional filters to use
nb_filters = 64
# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (3, 3)
# batch size
batch_size = 4
# number of classes (see labels above!)
num_classes = 5


# see the prev result
nb_train_samples, img_rows, img_cols, img_channels = (64, 1024, 1024, 3)
input_shape = (img_rows, img_cols, img_channels)


# model definition
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

# Train the network

In [ ]:
nb_epoch = 5

model.fit_generator(
    generator=trainDataGenerator(
       targetUrl, 
       batch_size
    ),
    steps_per_epoch=8,
    epochs=nb_epoch,
    validation_data=trainDataGenerator(
        targetUrlTestData, 
        batch_size
    ),
    verbose=1,
    validation_steps=4
)

number of images: 384
number of images: 48
Epoch 1/5
batch_X shape: (4, 1024, 1024, 3)
-1
first: 0, second: 4
datasets/streetview/3#000015_0.jpg has shape: (1024, 1024, 3)
label: 3
datasets/streetview/8#000010_4.jpg has shape: (1024, 1024, 3)
label: 8
datasets/streetview/6#000006_5.jpg has shape: (1024, 1024, 3)
label: 6
datasets/streetview/8#000016_2.jpg has shape: (1024, 1024, 3)
label: 8
batch_X shape: (4, 1024, 1024, 3), batch_y shape: (4,)
batch_X shape: (4, 1024, 1024, 3)
3
first: 4, second: 8
datasets/streetview/3#000011_4.jpg has shape: (1024, 1024, 3)
label: 3
datasets/streetview/8#000009_5.jpg has shape: (1024, 1024, 3)
label: 8
datasets/streetview/1#000006_5.jpg has shape: (1024, 1024, 3)
label: 1
datasets/streetview/8#000014_0.jpg has shape: (1024, 1024, 3)
label: 8
batch_X shape: (4, 1024, 1024, 3), batch_y shape: (4,)
batch_X shape: (4, 1024, 1024, 3)
7
first: 8, second: 12
datasets/streetview/1#000006_4.jpg has shape: (1024, 1024, 3)
label: 1
datasets/streetview/8#000009

[1, 8, 3, 6]

In [55]:
keras.utils.to_categorical([[1, 8, 3, 6 ]], num_classes=None)

array([[[0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0.]]], dtype=float32)

In [56]:
keras.utils.to_categorical([1, 8, 3, 6 ], num_classes=None)

array([[0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0.]], dtype=float32)

In [64]:
keras.utils.to_categorical([1, 2, 3 ], num_classes=None)

array([[0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]], dtype=float32)